In [1]:
import pickle as pkl
import pandas as pd
import numpy as np

In [2]:
with open("pot_dub.txt","r") as f:
    pot_dub = f.readlines()
    pot_dub = [char.strip() for char in pot_dub]
    remove_char = []
    for char in pot_dub:
        if char[-3:] == "(R)":
            remove_char.append(char[:-4])

In [3]:
len(remove_char)

29

In [4]:
# Load the character dataset
character_df = pd.read_csv("1.Dataset_files/CharacterWikis.csv")
character_df.Aliases = [eval(char) for char in character_df.Aliases]
character_df.Tokens = [eval(t) for t in character_df.Tokens]
character_df[character_df.Name == "Lily Potter"].iloc[1].Name = "Lily L. Potter"

# Remove the rows with characters
for idx,row in character_df.iterrows():
    if row.Name in remove_char:
        character_df.drop(idx,inplace=True)
        continue

# Reset the index
character_df.index = np.arange(len(character_df))

In [5]:
character_df

,Name,Link,WikiText,Tokens,Aliases
0,Vernon Dursley,/wiki/Vernon_Dursley,"Vernon Dursley was an English Muggle, husband ...","[vernon, dursley, english, muggle, husband, pe...","[Mr. Dursley, Vernon]"
1,Petunia Dursley,/wiki/Petunia_Dursley,Petunia Dursley (née Evans) (pre 1960 – pre 20...,"[petunia, dursley, née, evans, pre, pre, engli...","[Tuney, Petunia, Mrs. Dursley]"
2,Dudley Dursley,/wiki/Dudley_Dursley,Dudley Dursley (b. 23 June 1980)[1] was the Mu...,"[dudley, dursley, b, june, muggle, son, vernon...","[Diddy, Diddykins, Dudders, Dudley, Popkin, My..."
3,Lily Potter,/wiki/Lily_J._Potter,Lily J.[8] Potter (née Evans) (30 January[1] 1...,"[lily, j, potter, née, evans, january, october...","[Lily, Mrs. Potter]"
4,James Potter,/wiki/James_Potter_I,"England,[2] Great Britain Godric's Hollow, Wes...","[england, great, britain, godric, hollow, west...","[Prongs, James]"
...,...,...,...,...,...
673,Hugo Weasley,/wiki/Hugo_Granger-Weasley,Hugo Granger-Weasley was born around 2008 to a...,"[hugo, born, around, father, mother, making, t...",[Hugo]
674,Scorpius Malfoy,/wiki/Scorpius_Malfoy,Scorpius Hyperion Malfoy (b. c. 2006) was a Br...,"[scorpius, hyperion, malfoy, british, wizard, ...","[Scorpius, Scorpius the Dreadless, Scorpion Ki..."
675,Victoire Weasley,/wiki/Victoire_Weasley,Victoire Weasley (b. 2 May[1] 2000 or 2001)[2]...,"[victoire, weasley, b, may, witch, eldest, chi...",[Victoire]
676,Astoria Greengrass,/wiki/Astoria_Malfoy,Great Britain Astoria Malfoy[8] (née Greengras...,"[great, britain, astoria, malfoy, née, greengr...",[Astoria]


In [6]:
# Clean the aliases
# If alias is a name of a character, remove it

Names = character_df.Name.tolist()
for idx,row in character_df.iterrows():
    for alias in row.Aliases:
        if alias in Names:
            character_df.at[idx,"Aliases"].remove(alias)

# Readd the names to the aliases
for idx,row in character_df.iterrows():
    if row.Name not in row.Aliases:
        character_df.at[idx,"Aliases"].append(row.Name)

In [7]:
# Reverse traverse the aliases to remove duplicates
for i in range(len(character_df)-1,-1,-1):
    row = character_df.iloc[i]

    # Get all the aliases before the current row
    aliases = []
    for alias in character_df['Aliases'][:i]:
        aliases.extend(alias)

    # Remove the duplicates
    for alias in row.Aliases:
        if alias in aliases:
            character_df.at[i,"Aliases"].remove(alias)

In [8]:
character_df.to_csv("1.Dataset_files/CharacterWikis_clean.csv",index=False)